# Pokemon Classifier
A Convolutional Neural Network (CNN) designed to detect multiple Pokemon in a given image. The model is trained on a dataset containing various Pokemon characters and can identify multiple Pokemon in a single image.


In [ ]:
# Start off by randomly picking images from the training set and displaying them with a Unknown label at the bottom
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

training_directory = 'data/training_set'
testing_directory = 'data/testing_set'

#plot 5 Pokemon as subplots together, turn off axis, normal colors
fig, ax = plt.subplots(1, 5, figsize=(20, 20))
for i in range(5):
    random_pokemon = random.choice(os.listdir(training_directory))
    random_pokemon_image = random.choice(os.listdir(training_directory + '/' + random_pokemon))
    img = mpimg.imread(training_directory + '/' + random_pokemon + '/' + random_pokemon_image)
    ax[i].set_title(random_pokemon)
    ax[i].axis('off')
    ax[i].imshow(img)

In [12]:
from PIL import Image
#use sklearn to create  CNN
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

#create sklearn model to classify pokemon images
model = MLPClassifier(
    hidden_layer_sizes=(100, 100, 100),
    max_iter=500,
    alpha=0.0001,
    solver='sgd',
    verbose=10,
    random_state=21,
    tol=0.000000001
)

#loop through creating dataset that contains [label, path_to_image]
dataset = []
#test with 100 images
for i in range(100):
    random_pokemon = random.choice(os.listdir(training_directory))
    random_pokemon_image = random.choice(os.listdir(training_directory + '/' + random_pokemon))
    dataset.append([random_pokemon, training_directory + '/' + random_pokemon + '/' + random_pokemon_image])


# create x and y for training
x = []
y = []

for label, path in dataset:
    img = Image.open(path)
    img = img.resize((50, 50))
    img = np.array(img)
    img = img.flatten()
    x.append(img)
    y.append(label)

# convert x and y to numpy arrays
x = np.array(x)
y = np.array(y)

# split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

#reshape x_train and x_test
x_train = x_train.reshape(x_train.shape[0], 7500)
x_test = x_test.reshape(x_test.shape[0], 7500)







(3010000,)
(100,)


In [13]:






#fit model to training data
model.fit(x_train, y_train)

# attempt to make predictions


#show images with prediction and actual labels
#graph accuracy and loss


ValueError: Found input variables with inconsistent numbers of samples: [3010000, 100]